In [1]:
from qcutils.credential import load_provider
from qcutils.benchmarks import QASMBenchmark
from qcutils.pulse import CircAmpTimeSeries, BasisPulse
from qcutils.pulse.amptimeseries.metric import dist, min_circ_dist_in_list

from qiskit import QuantumCircuit, transpile, schedule, Aer
from qiskit.quantum_info import Operator
from qiskit.quantum_info import state_fidelity
import qiskit.quantum_info as qi
import numpy as np


provider = load_provider()
backend = provider.get_backend("ibm_lagos")

bm = QASMBenchmark(r"E:\Research\QASMBench", "small", num_qubits_list=list(range(8)), remove_final_measurements=True, do_transpile=False)

In [2]:
def transform_sx(circ, num_sx = None):
    from qiskit.circuit import CircuitInstruction, Instruction, ParameterExpression
    import random

    flag_random = False
    if num_sx == None:
        flag_random = True

    def transformed_sx(qubit, num_sx):
        if num_sx % 4 == 1:
            sx_1 = QuantumCircuit(circ.num_qubits)
            sx_1.sx(qubit)
            sx_1_circ_instructions = sx_1.data
            return sx_1_circ_instructions
        elif num_sx % 4 == 2:
            sx_2 = QuantumCircuit(circ.num_qubits)
            sx_2.rz(-np.pi/2, qubit)
            sx_2.sx(qubit)
            sx_2.rz(np.pi/2, qubit)
            sx_2.sx(qubit)
            sx_2.rz(-np.pi/2, qubit)
            sx_2_circ_instructions = sx_2.data
            return sx_2_circ_instructions
        elif num_sx % 4 == 3:
            sx_3 = QuantumCircuit(circ.num_qubits)
            sx_3.rz(-np.pi, qubit)
            sx_3.sx(qubit)
            sx_3.rz(np.pi/2, qubit)
            sx_3.sx(qubit)
            sx_3.sx(qubit)
            sx_3.rz(-np.pi/2, qubit)
            sx_3_circ_instructions = sx_3.data
            return sx_3_circ_instructions
        elif num_sx % 4 == 0:
            sx_4 = QuantumCircuit(circ.num_qubits)
            sx_4.rz(-np.pi, qubit)
            sx_4.sx(qubit)
            sx_4.sx(qubit)
            sx_4.rz(np.pi/2, qubit)
            sx_4.sx(qubit)
            sx_4.rz(-np.pi/2, qubit)
            sx_4.sx(qubit)
            sx_4.rz(-np.pi/2, qubit)
            sx_4_circ_instructions = sx_4.data
            return sx_4_circ_instructions

    circ_instructions = []
    for circ_instruction in circ.data:
        if circ_instruction.operation.name == "sx":
            qubit_idx = circ.find_bit(circ_instruction.qubits[0]).index
            # print(qubit_idx)
            # print(transformed_sx(qubit_idx, 2))
            if flag_random == None:
                num_sx = random.randint(1, 4)
            circ_instructions += transformed_sx(qubit_idx, num_sx)
        else:
            circ_instructions.append(circ_instruction)
    # return circ_instructions

    # transformed_circ = QuantumCircuit(circ.num_qubits, circ.num_clbits)
    return QuantumCircuit.from_instructions(circ_instructions, qubits=circ.qubits, clbits=circ.clbits)

In [3]:
simulator = Aer.get_backend('aer_simulator')

print("Fidelity")
count = 0
for circ_name, circ in bm.items():
    try:
        circ = transpile(circ, backend)
        dm1 = qi.Statevector.from_instruction(circ)
        circ = transpile(transform_sx(circ), backend, optimization_level=0)
        dm2 = qi.Statevector.from_instruction(circ)
        
        print(f"{circ_name}: {state_fidelity(dm1, dm2)}")
        count += 1
    except:
        print(f"{circ_name} raises exception")

print(f"Number of success circuits: {count}/{len(bm)}")

Fidelity
deutsch_n2: 1.0000000000000009
dnn_n2: 1.0000000000000422
grover_n2: 1.0000000000000022
ipea_n2 raises exception
iswap_n2: 1.0000000000000022
quantumwalks_n2: 1.0000000000000067
basis_change_n3: 1.0000000000000067
fredkin_n3: 1.0000000000000013
linearsolver_n3: 1.000000000000004
qaoa_n3: 1.0000000000000049
teleportation_n3: 1.0000000000000018
toffoli_n3: 1.0000000000000018
wstate_n3: 1.0000000000000013
adder_n4: 1.0
basis_trotter_n4: 1.0000000000003233
bell_n4: 1.0000000000000093
cat_state_n4: 1.0000000000000004
hs4_n4: 1.0000000000000004
inverseqft_n4 raises exception
qft_n4: 1.0000000000000022
qrng_n4: 1.0000000000000022
variational_n4: 1.0000000000000058
vqe_n4: 1.0000000000000093
vqe_uccsd_n4: 1.0000000000000213
error_correctiond3_n5: 1.000000000000004
lpn_n5: 1.0000000000000009
pea_n5: 1.000000000000004
qec_en_n5: 1.0000000000000036
qec_sm_n5 raises exception
shor_n5 raises exception
qaoa_n6: 1.0000000000000595
simon_n6: 1.0000000000000022
vqe_uccsd_n6: 1.0000000000001878

# Duration

In [3]:
print("Benchmark\tDuration Ratio (num_sx)")
print("\t\t\t2\t3\t4")
count = 0
for circ_name, circ in bm.items():
    circ = transpile(circ, backend, seed_transpiler=0)
    try:
        duration_original = schedule(circ, backend).duration
        duration_list = [schedule(transpile(transform_sx(circ, num_sx), backend, optimization_level=0), backend).duration for num_sx in range(2, 5)]
        # num_sx_list = [transformed_circ.size(filter_function=lambda x: x.operation.name == "sx" and x.opeartion.qubit == i) for i in range(backend.configuration().n_qubits)]
        # duration = schedule(transformed_circ, backend).duration
        print(f"{circ_name:24}{duration_list[0]/duration_original:.3f}\t{duration_list[1]/duration_original:.2f}\t{duration_list[2]/duration_original:.2f}")
        count += 1
    except:
        print(f"{circ_name:24} raises exception")

print(f"Number of success circuits: {count}/{len(bm)}")

Benchmark	Duration Ratio (num_sx)
			2	3	4
deutsch_n2              1.110	1.22	1.33
dnn_n2                  1.088	1.18	1.27
grover_n2               1.085	1.17	1.25
ipea_n2                  raises exception
iswap_n2                1.085	1.17	1.25
quantumwalks_n2         1.141	1.28	1.42
basis_change_n3         1.118	1.24	1.35
fredkin_n3              1.007	1.01	1.02
linearsolver_n3         1.109	1.22	1.33
qaoa_n3                 1.037	1.07	1.11
teleportation_n3        1.076	1.15	1.23
toffoli_n3              1.025	1.05	1.07
wstate_n3               1.020	1.04	1.06
adder_n4                1.014	1.03	1.04
basis_trotter_n4        1.069	1.14	1.21
bell_n4                 1.164	1.33	1.49
cat_state_n4            1.037	1.07	1.11
hs4_n4                  1.138	1.28	1.41
inverseqft_n4            raises exception
qft_n4                  1.012	1.02	1.04
qrng_n4                 2.000	3.00	4.00
variational_n4          1.053	1.11	1.16
vqe_n4                  1.112	1.22	1.34
vqe_uccsd_n4            1.023	1.0